In [1]:
%pip install arabic-stopwords
%pip install gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 8.1 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [10]:
!gdown "1n6IHWly66qSJfxrDSaIBcqm05jo-v6mt"
!gdown "1zToBjksCyT4DYAZ-0Oqj52nEXk0aqtdm"
!gdown "1wKZjFyWcfsTveGpuRwXHqmfFW8Zu8cIe"
!gdown "10S8tYhh1jd5VyQD7Rbw2QVBXIfw3hd1H"
!gdown "1Fs0PTy_xPsoX5bg4QTCrX6JAaQCHRzis"
!gdown "1JLa-ELhUskQINi0syf3YJWBX8jcCXdn4"

Downloading...
From: https://drive.google.com/uc?id=1n6IHWly66qSJfxrDSaIBcqm05jo-v6mt
To: /kaggle/working/rs_train_embeddings.pkl
100%|███████████████████████████████████████| 49.5M/49.5M [00:00<00:00, 139MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zToBjksCyT4DYAZ-0Oqj52nEXk0aqtdm
To: /kaggle/working/rs_test_embeddings.pkl
100%|███████████████████████████████████████| 12.4M/12.4M [00:00<00:00, 103MB/s]
Downloading...
From: https://drive.google.com/uc?id=1wKZjFyWcfsTveGpuRwXHqmfFW8Zu8cIe
To: /kaggle/working/ls_train_embeddings.pkl
100%|███████████████████████████████████████| 49.5M/49.5M [00:00<00:00, 125MB/s]
Downloading...
From: https://drive.google.com/uc?id=10S8tYhh1jd5VyQD7Rbw2QVBXIfw3hd1H
To: /kaggle/working/ls_test_embeddings.pkl
100%|███████████████████████████████████████| 12.4M/12.4M [00:00<00:00, 122MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Fs0PTy_xPsoX5bg4QTCrX6JAaQCHRzis
To: /kaggle/working/train.pkl
100%|█████████████████████████████████████

In [4]:
import pandas as pd
import nltk
import string
import re
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.isri import ISRIStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
from sklearn.svm import SVC
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split, GridSearchCV
import arabicstopwords.arabicstopwords as ast
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import torch
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from collections import OrderedDict
import matplotlib.pyplot as plt
from datasets import load_dataset
from sklearn.preprocessing import OneHotEncoder

In [9]:
dataset = load_dataset('emotone_ar')

Generating train split:   0%|          | 0/10065 [00:00<?, ? examples/s]

Dataset emotone_ar downloaded and prepared to /root/.cache/huggingface/datasets/emotone_ar/default/0.0.0/36bfa7e8bdf640ccb3d5701933cfde2fa7051f22881cfab2962122771d32c430. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
def normalizeArabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)

    return(text)

stop_words = ast.stopwords_list()

def remove_stop_words(text):
    return ' '.join(word for word in str(text).split() if word not in stop_words)

arabic_punctuations = '''`÷×؛<>_():*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


# snow_stemmer = SnowballStemmer(language='arabic')
isri_stemmer = ISRIStemmer()

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
def stemming(text):
  x = w_tokenizer.tokenize(text)
  # return [snow_stemmer.stem(w) for w in x]
  return [isri_stemmer.stem(w) for w in x]

def prepare_text(text):
  text = normalizeArabic(text)
  text = remove_stop_words(text)
  text = remove_punctuations(text)
  text = stemming(text)
  return text

In [ ]:
df = pd.DataFrame(dataset['train'])

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df['label'].value_counts()

In [ ]:
g = df.groupby('label')
df_balanced = g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True))
df_balanced = df_balanced.sample(frac=1).reset_index(drop=True)

In [ ]:
df_balanced['original'] = df_balanced['tweet']
df_balanced['data'] = df_balanced['original'].apply(prepare_text)
df_balanced = df_balanced.drop(['tweet'], axis=1)

In [ ]:
df_balanced['label'].value_counts()

In [ ]:
df['label'].plot.hist()

In [ ]:
df_balanced['label'].plot.hist()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df_balanced['data'], df_balanced['label'], test_size=0.2, stratify=df_balanced['label'], random_state=42)

In [ ]:
encoder = OneHotEncoder()

y_train = encoder.fit_transform(y_train.values.reshape(-1,1)).toarray()
y_val = encoder.transform(y_val.values.reshape(-1,1)).toarray()

In [ ]:
def prepare_tokenization(train, val, test=None, pad=500):
  tokenizer = Tokenizer(oov_token='<UNK>')
  tokenizer.fit_on_texts(train)
  tokenizer.word_index['<PAD>'] = 0
  train = tokenizer.texts_to_sequences(train)
  val = tokenizer.texts_to_sequences(val)
  if not pad==False:
    train = sequence.pad_sequences(train, maxlen=pad)
    val = sequence.pad_sequences(val, maxlen=pad)
  if not type(test)==type(None):
    test = tokenizer.texts_to_sequences(test)
    if not pad==False:
      test = sequence.pad_sequences(test, maxlen=pad)
  v_size = len(tokenizer.word_index)
  print("Vocabulary size={}".format(v_size))
  print("Number of Documents={}".format(tokenizer.document_count))
  if not type(test)==type(None):
    return v_size, train, val, test
  else:
    return v_size, train, val

In [ ]:
VOCAB_SIZE, train_sequences, val_sequences = prepare_tokenization(X_train, X_val, pad=False)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

train_lens = [len(s) for s in train_sequences]
plt.hist(train_lens)

In [ ]:
max_length = max(train_lens)

In [ ]:
max_length

In [ ]:
X_train = sequence.pad_sequences(train_sequences, maxlen=max_length)
X_val = sequence.pad_sequences(val_sequences, maxlen=max_length)
X_train.shape, X_val.shape

In [11]:
with open('/kaggle/working/train.pkl', 'rb') as f:
    train = pickle.load(f)

with open('/kaggle/working/test.pkl', 'rb') as f:
    test = pickle.load(f)

with open('/kaggle/working/rs_train_embeddings.pkl', 'rb') as f:
    rs_train_embeddings = pickle.load(f)
    
with open('/kaggle/working/rs_test_embeddings.pkl', 'rb') as f:
    rs_test_embeddings = pickle.load(f)
    
with open('/kaggle/working/ls_train_embeddings.pkl', 'rb') as f:
    ls_train_embeddings = pickle.load(f)
    
with open('/kaggle/working/ls_test_embeddings.pkl', 'rb') as f:
    ls_test_embeddings = pickle.load(f)

In [17]:
encoder = OneHotEncoder()

y_train = encoder.fit_transform(train['label'].values.reshape(-1,1)).toarray()
y_test = encoder.transform(test['label'].values.reshape(-1,1)).toarray()

In [15]:
train['label'].values.shape

(8052,)

In [8]:
rs_train_embeddings.shape

(8052, 768)

In [18]:
X_train_gru_global = ls_train_embeddings.reshape(ls_train_embeddings.shape[0], 1, ls_train_embeddings.shape[1]) # reshape input to allow for GRU
X_test_gru_global = ls_test_embeddings.reshape(ls_test_embeddings.shape[0], 1, ls_test_embeddings.shape[1]) # reshape input to allow for GRU

In [19]:
BATCH_SIZE = 128
EPOCHS = 50
EMBED_SIZE = 300
LEARNING_RATE =  0.001

early_stopping_monitor = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    min_delta=0,
    patience=10,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True)

In [47]:
model = Sequential()
# model.add(Embedding(VOCAB_SIZE, EMBED_SIZE, input_length=max_length, trainable=True))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(512, input_shape=(X_train_gru_global.shape[1], X_train_gru_global.shape[2]), return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(512, return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, return_sequences=False)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
# model.summary()

history = model.fit(X_train_gru_global, np.asarray(y_train), validation_data=(X_test_gru_global, np.asarray(y_test)), batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[early_stopping_monitor])

Epoch 1/50
63/63 [==============================] - 24s 167ms/step - loss: 1.6786 - accuracy: 0.3830 - val_loss: 1.1080 - val_accuracy: 0.6279
Epoch 2/50
63/63 [==============================] - 9s 136ms/step - loss: 1.1860 - accuracy: 0.6179 - val_loss: 0.9855 - val_accuracy: 0.6786
Epoch 3/50
63/63 [==============================] - 9s 135ms/step - loss: 1.0038 - accuracy: 0.6872 - val_loss: 0.9201 - val_accuracy: 0.7074
Epoch 4/50
63/63 [==============================] - 8s 134ms/step - loss: 0.8974 - accuracy: 0.7255 - val_loss: 0.9357 - val_accuracy: 0.6935
Epoch 5/50
63/63 [==============================] - 8s 131ms/step - loss: 0.8224 - accuracy: 0.7516 - val_loss: 0.9274 - val_accuracy: 0.7104
Epoch 6/50
63/63 [==============================] - 9s 135ms/step - loss: 0.7268 - accuracy: 0.7788 - val_loss: 1.0262 - val_accuracy: 0.6935
Epoch 7/50
63/63 [==============================] - 8s 132ms/step - loss: 0.6761 - accuracy: 0.7953 - val_loss: 0.9936 - val_accuracy: 0.7139
Epoch

In [34]:
model1 = Sequential()
# model1.add(Embedding(VOCAB_SIZE, EMBED_SIZE, input_length=max_length, trainable=True))
model1.add(tf.keras.layers.GRU(512, input_shape=(X_train_gru_global.shape[1], X_train_gru_global.shape[2]), return_sequences=True))
model1.add(tf.keras.layers.GRU(256, return_sequences=True))
model1.add(tf.keras.layers.GRU(128, return_sequences=False))
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(64, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(8, activation='softmax'))
opt1 = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model1.compile(loss='categorical_crossentropy', optimizer=opt1, metrics=['accuracy'])
# model1.summary()

history = model1.fit(X_train_gru_global, np.asarray(y_train), validation_data=(X_test_gru_global, np.asarray(y_test)), batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[early_stopping_monitor])

Epoch 1/50
63/63 [==============================] - 10s 60ms/step - loss: 1.3008 - accuracy: 0.5592 - val_loss: 0.9419 - val_accuracy: 0.6850
Epoch 2/50
63/63 [==============================] - 3s 45ms/step - loss: 0.8683 - accuracy: 0.7103 - val_loss: 0.9004 - val_accuracy: 0.6900
Epoch 3/50
63/63 [==============================] - 3s 42ms/step - loss: 0.7422 - accuracy: 0.7530 - val_loss: 0.8578 - val_accuracy: 0.7154
Epoch 4/50
63/63 [==============================] - 3s 40ms/step - loss: 0.6514 - accuracy: 0.7840 - val_loss: 0.9396 - val_accuracy: 0.7049
Epoch 5/50
63/63 [==============================] - 3s 44ms/step - loss: 0.5649 - accuracy: 0.8128 - val_loss: 0.9528 - val_accuracy: 0.6950
Epoch 6/50
63/63 [==============================] - 3s 40ms/step - loss: 0.5157 - accuracy: 0.8291 - val_loss: 1.0038 - val_accuracy: 0.6985
Epoch 7/50
63/63 [==============================] - 3s 41ms/step - loss: 0.4547 - accuracy: 0.8502 - val_loss: 1.1288 - val_accuracy: 0.6870
Epoch 8/50
6

In [37]:
model2 = Sequential()
# model2.add(Embedding(VOCAB_SIZE, EMBED_SIZE, input_length=max_length, trainable=True))
model2.add(tf.keras.layers.GRU(128, input_shape=(X_train_gru_global.shape[1], X_train_gru_global.shape[2]), return_sequences=False))
model2.add(Dropout(0.5))
model2.add(Flatten())
model2.add(Dense(8, activation='softmax'))
opt2 = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model2.compile(loss='categorical_crossentropy', optimizer=opt2, metrics=['accuracy'])
# model2.summary()

history = model2.fit(X_train_gru_global, np.asarray(y_train), validation_data=(X_test_gru_global, np.asarray(y_test)), batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[early_stopping_monitor])

Epoch 1/50
63/63 [==============================] - 3s 18ms/step - loss: 1.3672 - accuracy: 0.5277 - val_loss: 0.9507 - val_accuracy: 0.6816
Epoch 2/50
63/63 [==============================] - 1s 9ms/step - loss: 0.9573 - accuracy: 0.6824 - val_loss: 0.8687 - val_accuracy: 0.7044
Epoch 3/50
63/63 [==============================] - 1s 8ms/step - loss: 0.8535 - accuracy: 0.7084 - val_loss: 0.8282 - val_accuracy: 0.7233
Epoch 4/50
63/63 [==============================] - 1s 9ms/step - loss: 0.7751 - accuracy: 0.7327 - val_loss: 0.8178 - val_accuracy: 0.7168
Epoch 5/50
63/63 [==============================] - 1s 9ms/step - loss: 0.7262 - accuracy: 0.7512 - val_loss: 0.8098 - val_accuracy: 0.7149
Epoch 6/50
63/63 [==============================] - 1s 9ms/step - loss: 0.6793 - accuracy: 0.7647 - val_loss: 0.8038 - val_accuracy: 0.7253
Epoch 7/50
63/63 [==============================] - 1s 9ms/step - loss: 0.6511 - accuracy: 0.7729 - val_loss: 0.8140 - val_accuracy: 0.7168
Epoch 8/50
63/63 [=

In [25]:
model3 = Sequential()
# model3.add(Embedding(VOCAB_SIZE, EMBED_SIZE, input_length=max_length, trainable=True))
model3.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, input_shape=(X_train_gru_global.shape[1], X_train_gru_global.shape[2]), return_sequences=False)))
model3.add(Dropout(0.5))
model3.add(Flatten())
model3.add(Dense(8, activation='softmax'))
opt3 = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model3.compile(loss='categorical_crossentropy', optimizer=opt3, metrics=['accuracy'])
# model3.summary()

history = model3.fit(X_train_gru_global, np.asarray(y_train), validation_data=(X_test_gru_global, np.asarray(y_test)), batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[early_stopping_monitor])

Epoch 1/50
63/63 [==============================] - 6s 26ms/step - loss: 1.2639 - accuracy: 0.5637 - val_loss: 0.8934 - val_accuracy: 0.6965
Epoch 2/50
63/63 [==============================] - 1s 14ms/step - loss: 0.8690 - accuracy: 0.6966 - val_loss: 0.8669 - val_accuracy: 0.6975
Epoch 3/50
63/63 [==============================] - 1s 14ms/step - loss: 0.7707 - accuracy: 0.7310 - val_loss: 0.8163 - val_accuracy: 0.7119
Epoch 4/50
63/63 [==============================] - 1s 14ms/step - loss: 0.7122 - accuracy: 0.7474 - val_loss: 0.8253 - val_accuracy: 0.7114
Epoch 5/50
63/63 [==============================] - 1s 13ms/step - loss: 0.6554 - accuracy: 0.7676 - val_loss: 0.8081 - val_accuracy: 0.7228
Epoch 6/50
63/63 [==============================] - 1s 13ms/step - loss: 0.6187 - accuracy: 0.7784 - val_loss: 0.8139 - val_accuracy: 0.7203
Epoch 7/50
63/63 [==============================] - 1s 15ms/step - loss: 0.5642 - accuracy: 0.7983 - val_loss: 0.8294 - val_accuracy: 0.7119
Epoch 8/50
63

In [41]:
model4 = Sequential()
# model4.add(Embedding(VOCAB_SIZE, EMBED_SIZE, input_length=max_length, trainable=True))
model4.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, input_shape=(X_train_gru_global.shape[1], X_train_gru_global.shape[2]), return_sequences=False)))
model4.add(Dropout(0.5))
model4.add(Dense(128, activation='relu'))
model4.add(Dropout(0.5))
model4.add(Dense(8, activation='softmax'))
opt4 = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model4.compile(loss='categorical_crossentropy', optimizer=opt4, metrics=['accuracy'])
# model4.summary()

history = model4.fit(X_train_gru_global, np.asarray(y_train), validation_data=(X_test_gru_global, np.asarray(y_test)), batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[early_stopping_monitor])

Epoch 1/50
63/63 [==============================] - 7s 39ms/step - loss: 1.5294 - accuracy: 0.4692 - val_loss: 0.9906 - val_accuracy: 0.6617
Epoch 2/50
63/63 [==============================] - 1s 24ms/step - loss: 1.0677 - accuracy: 0.6328 - val_loss: 0.8822 - val_accuracy: 0.6945
Epoch 3/50
63/63 [==============================] - 1s 24ms/step - loss: 0.9490 - accuracy: 0.6759 - val_loss: 0.8622 - val_accuracy: 0.7009
Epoch 4/50
63/63 [==============================] - 2s 25ms/step - loss: 0.8895 - accuracy: 0.6940 - val_loss: 0.8379 - val_accuracy: 0.7134
Epoch 5/50
63/63 [==============================] - 1s 23ms/step - loss: 0.8279 - accuracy: 0.7199 - val_loss: 0.8199 - val_accuracy: 0.7188
Epoch 6/50
63/63 [==============================] - 2s 25ms/step - loss: 0.7758 - accuracy: 0.7276 - val_loss: 0.8199 - val_accuracy: 0.7213
Epoch 7/50
63/63 [==============================] - 2s 26ms/step - loss: 0.7525 - accuracy: 0.7377 - val_loss: 0.8240 - val_accuracy: 0.7223
Epoch 8/50
63

In [28]:
predictions = model.predict(X_test_gru_global)
print(classification_report(y_test.argmax(axis=1), predictions.argmax(axis=1)))

63/63 [==============================] - 3s 10ms/step
              precision    recall  f1-score   support

           0       0.70      0.73      0.71       307
           1       0.64      0.84      0.73       276
           2       0.62      0.57      0.59       268
           3       0.62      0.62      0.62       258
           4       0.78      0.78      0.78       250
           5       0.77      0.87      0.82       194
           6       0.70      0.41      0.52       201
           7       0.93      0.88      0.90       259

    accuracy                           0.72      2013
   macro avg       0.72      0.71      0.71      2013
weighted avg       0.72      0.72      0.71      2013



In [29]:
predictions1 = model1.predict(X_test_gru_global)
print(classification_report(y_test.argmax(axis=1), predictions1.argmax(axis=1)))

63/63 [==============================] - 1s 6ms/step
              precision    recall  f1-score   support

           0       0.64      0.91      0.75       307
           1       0.65      0.82      0.73       276
           2       0.58      0.66      0.62       268
           3       0.61      0.57      0.59       258
           4       0.85      0.63      0.72       250
           5       0.88      0.76      0.82       194
           6       0.72      0.39      0.51       201
           7       0.96      0.84      0.89       259

    accuracy                           0.71      2013
   macro avg       0.74      0.70      0.70      2013
weighted avg       0.73      0.71      0.71      2013



In [30]:
predictions2 = model2.predict(X_test_gru_global)
print(classification_report(y_test.argmax(axis=1), predictions2.argmax(axis=1)))

63/63 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.67      0.86      0.75       307
           1       0.70      0.81      0.75       276
           2       0.67      0.59      0.63       268
           3       0.64      0.60      0.62       258
           4       0.79      0.74      0.77       250
           5       0.83      0.82      0.82       194
           6       0.62      0.48      0.54       201
           7       0.92      0.88      0.90       259

    accuracy                           0.73      2013
   macro avg       0.73      0.72      0.72      2013
weighted avg       0.73      0.73      0.73      2013



In [31]:
predictions3 = model3.predict(X_test_gru_global)
print(classification_report(y_test.argmax(axis=1), predictions3.argmax(axis=1)))

63/63 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.67      0.83      0.74       307
           1       0.68      0.79      0.73       276
           2       0.67      0.59      0.63       268
           3       0.71      0.53      0.61       258
           4       0.74      0.82      0.78       250
           5       0.82      0.86      0.84       194
           6       0.59      0.46      0.52       201
           7       0.93      0.88      0.91       259

    accuracy                           0.73      2013
   macro avg       0.73      0.72      0.72      2013
weighted avg       0.72      0.73      0.72      2013



In [48]:
predictions4 = model4.predict(X_test_gru_global)
print(classification_report(y_test.argmax(axis=1), predictions4.argmax(axis=1)))

63/63 [==============================] - 0s 4ms/step
              precision    recall  f1-score   support

           0       0.67      0.88      0.76       307
           1       0.70      0.79      0.74       276
           2       0.65      0.62      0.63       268
           3       0.73      0.56      0.63       258
           4       0.78      0.77      0.78       250
           5       0.78      0.90      0.84       194
           6       0.65      0.42      0.51       201
           7       0.92      0.88      0.90       259

    accuracy                           0.73      2013
   macro avg       0.74      0.73      0.72      2013
weighted avg       0.74      0.73      0.73      2013

